# Cleaning and exploration

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import missingno as msno
import seaborn as sns
from wordcloud import WordCloud, STOPWORDS
import gc

import sys
sys.path.append("Modules/")

import nlp_module as nlp

ModuleNotFoundError: No module named 'pandas'

In [ ]:
sns.set_theme(style="ticks",
              palette="Pastel2")

## 1) Exploring the books datasets

In [ ]:
# importing different files and 
# concatenating in one dataset
filepath = 'Datasets/'

all_files = glob.glob(filepath + "books/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename)
    li.append(df)

merged_books = pd.concat(li,
                        axis=0,
                        ignore_index=True)
merged_books.head()

In [ ]:
# Identify missing values
msno.matrix(merged_books)

In [ ]:
merged_books["NumberOfPages"] = merged_books["PagesNumber"].fillna(0)\
                              + merged_books["pagesNumber"].fillna(0)

In [ ]:
# deleting columns about publish day, month and ratingdists
merged_books.drop(["PublishMonth", "PublishDay",
                   "RatingDist5", "RatingDist4",
                   "RatingDist3", "RatingDist3",
                   "RatingDist2", "RatingDist1",
                   "RatingDistTotal", "Count of text reviews",
                   "PagesNumber", "pagesNumber"], axis=1,
                   inplace=True)

In [ ]:
merged_books.head()

In [ ]:
merged_books.dropna(inplace=True)

In [ ]:
msno.matrix(merged_books)

In [ ]:
merged_books.shape

In [ ]:
merged_books.isnull().mean() * 100

### a) Numerical variables

In this part, we explore informations about numerical variables. These variables are the year of publication, the average ratings of each books, the number of ratings, the number of pages and the number of reviews.

In [ ]:
sns.boxplot(x="PublishYear",
            data=merged_books)
plt.show()

In [ ]:
# dropping data where year of publication is 
# less than 1820 & more than 2019
ind = merged_books.loc[(merged_books["PublishYear"] <= 1820)\
                       | (merged_books["PublishYear"] >= 2019)].index\
                       .to_list()
merged_books.drop(index=ind,
                  inplace=True)

In [ ]:
fig, axes = plt.subplots(1, 2,
                         figsize=(15, 5),
                         sharex=True)
fig.suptitle('Description of the year of publication of each book')

sns.histplot(ax=axes[0],
             x="PublishYear",
             data=merged_books,
             kde=True)
axes[0].set_title("Distribution of the year of publication")

sns.boxplot(ax=axes[1],
            x="PublishYear",
            data=merged_books)
axes[1].set_title("Boxplot of the year of publication")

plt.show()

After cleaning, only books with a year of publication between 1820 and 2019 were conserved. The majority of the books had been published between 1980 and 2019. The average is around 2000. 

In [ ]:
fig, axes = plt.subplots(1, 2,
                         figsize=(15, 5),
                         sharex=True)
fig.suptitle('Description of the number of reviews of each book')

sns.histplot(ax=axes[0],
             x="CountsOfReview",
             data=merged_books, kde=True)
axes[0].set_title("Distribution of the number of reviews")

sns.boxplot(ax=axes[1],
            x="CountsOfReview",
            data=merged_books)
axes[1].set_title("Boxplot of the number of reviews")

plt.show()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15, 5), sharex=True)
fig.suptitle('Description of the average ratings of each book')

sns.histplot(ax=axes[0],
             x="Rating",
             data=merged_books,
             kde=True)
axes[0].set_title("Distribution of the average ratings")

sns.boxplot(ax=axes[1],
            x="Rating",
            data=merged_books)
axes[1].set_title("Boxplot of the average ratings")

plt.show()

### b) Categorical variables

In [ ]:
sns.countplot(y="Authors",
              data=merged_books,
              order=merged_books["Authors"]\
              .value_counts()\
              .index[:15])

plt.title("Top 15 of the most read authors")
plt.show()

In [ ]:
sns.countplot(y="Publisher",
              data=merged_books,
              order=merged_books["Publisher"]\
              .value_counts()\
              .index[:15])
plt.title("Top 15 of the most present publishers")
plt.show()

In [ ]:
sns.countplot(y="Language",
              data=merged_books,
              order=merged_books["Language"]\
              .value_counts()\
              .index[:15])
plt.title("Top 15 of the most represented languages")
plt.show()

In [ ]:
lang = {
    "en-US" : "eng",
    "en-GB" : "eng",
    "en-CA" : "eng"
}

In [2]:
for key, value in lang.items():
    merged_books["Language"]\
    .loc[merged_books["Language"]\
    .str.contains(key, 
                  case=False,
                  regex=False)] = value

NameError: name 'lang' is not defined

In [ ]:
sns.countplot(y="Language",
              data=merged_books,
              order=merged_books["Language"]\
              .value_counts()\
              .index[:15])
plt.title("Top 15 of the most represented languages")
plt.show()

In [ ]:
merged_books_en = merged_books[merged_books["Language"] == "eng"]
merged_books_en.head()

### c) Description of books analysis

In [ ]:
desc = merged_books_en[["Id", "Description"]]
desc.head(3)

In [ ]:
desc["Description"] = nlp.normalize_corpus(desc["Description"])

In [ ]:
desc.head(3)

In [ ]:
desc.to_csv("Datasets/description.csv")
desc.to_parquet("Datasets/description.parquet.gzip",
                compression="gzip",
                index=False)

In [ ]:
stopwords = set(STOPWORDS)
wordcloud = WordCloud(width = 800, height = 1000,
                max_words=1000,
                background_color ='white',
                stopwords = stopwords,
                min_font_size = 12,
                colormap="Set2").generate(str(desc["Description"]))
 
# plot the WordCloud image                      
plt.figure(figsize = (8, 8),
           facecolor = None)
plt.imshow(wordcloud)
plt.axis("off")
plt.tight_layout(pad = 0)

In [ ]:
merged_books_en.drop("Description",
                     axis=1,
                     inplace=True)
data = pd.merge(merged_books_en, desc, on="Id")
data.head(3)

In [ ]:
data.drop("Language",
          axis=1,
          inplace=True)

In [ ]:
data.to_csv("Datasets/books_data_cleaning.csv")
data.to_parquet("Datasets/books_data_cleaning.parquet.gzip",
                compression="gzip",
                index=False)

In [ ]:
del data
del merged_books
del merged_books_en
gc.collect()

## 2) Exploring the users datasets

In [ ]:
book_data = pd.read_parquet("Datasets/books_data_cleaning.parquet.gzip")

In [ ]:
# importing different files and 
# concatenating in one dataset
filepath = 'Datasets/'

all_files = glob.glob(filepath + "users_ratings/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename)
    li.append(df)

merged_users = pd.concat(li,
                        axis=0,
                        ignore_index=True)
merged_users.head()

In [ ]:
# Identify missing values
msno.matrix(merged_users)

In [ ]:
merged_users["Rating"].unique()

In [ ]:
new_ratings ={
    "This user doesn't have any rating" : np.nan,
    "did not like it" : 1,
    "it was ok" : 2,
    "liked it" : 3,
    "really liked it" : 4,
    "it was amazing" : 5
}

merged_users["Rating"] = merged_users["Rating"].map(new_ratings)

In [ ]:
merged_users.columns = ['User_Id', "Name", "Rating"]

In [ ]:
# Identify missing values
msno.matrix(merged_users)

In [ ]:
merged_users.dropna(inplace=True)

In [ ]:
sns.countplot(y="User_Id",
              data=merged_users,
              order=merged_users["User_Id"].value_counts().index[:15])
plt.title("Top 15 of the users that made the most ratings")
plt.show()

In [ ]:
sns.countplot(y="Name",
              data=merged_users,
              order=merged_users["Name"].value_counts().index[:15])
plt.ylabel("Title of the book")
plt.title("Top 15 of the books that had the most ratings")
plt.show()

In [ ]:
book_data = book_data[["Id","Name", "Authors"]]

In [ ]:
rating_merged = pd.merge(merged_users, book_data, on = "Name")
rating_merged.head()

In [ ]:
rating_merged.drop(["Name", "Authors"],
                    axis=1, 
                    inplace=True, )

In [ ]:
rating_merged.to_csv("Datasets/users_data_cleaning.csv")
rating_merged.to_parquet("Datasets/users_data_cleaning.parquet.gzip",
                compression="gzip",
                index=False)